# Interactive time-gating

## Preliminaries

### Imports

In [ ]:
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from ipywidgets import interact, interactive, IntSlider, IntText, Dropdown, fixed, FloatSlider

import numpy as np
import pandas
from scipy.ndimage import convolve1d
from scipy import signal, special

from PyDynamic.uncertainty.propagate_DFT import GUM_iDFT, GUM_DFT, DFT2AmpPhase, AmpPhase2DFT
from PyDynamic.misc import complex_2_real_imag as c2ri
from PyDynamic.misc import real_imag_2_complex as ri2c
from PyDynamic.misc.tools import shift_uncertainty, trimOrPad

import interactive_gating_with_unc_utils as utils

### Compare different available datasets

In [ ]:
# main empirical data (Type A, but only very few samples)
f_emp, s11_mag_emp, s11_phase_emp, s11_mag_unc_emp, s11_phase_unc_emp = utils.load_data("empirical_cov", return_mag_phase=True, return_full_cov=False)

# mag-phase-diag only (Type B)
f_old, s11_mag_old, s11_phase_old, s11_mag_unc_old, s11_phase_unc_old = utils.load_data("diag_only", return_mag_phase=True, return_full_cov=False)

# simulated data (no unc)
f_sim, s11_mag_sim, s11_phase_sim, s11_mag_unc_sim, s11_phase_unc_sim = utils.load_data("simulated", return_mag_phase=True, return_full_cov=False)

In [ ]:
# visualize raw input in the frequency domain
fig_comp_raw, ax_comp_raw = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_comp_raw[0].plot(f_emp, s11_mag_emp, label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[0].plot(f_old, s11_mag_old, label="diag only (Type B)", color="tab:red")
ax_comp_raw[0].plot(f_sim, s11_mag_sim, label="simulated", color="tab:blue")

ax_comp_raw[1].semilogy(f_emp, s11_mag_unc_emp, label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[1].semilogy(f_old, s11_mag_unc_old, label="diag only (Type B)", color="tab:red")

ax_comp_raw[2].plot(f_emp, np.rad2deg(s11_phase_emp), label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[2].plot(f_old, np.rad2deg(s11_phase_old), label="diag only (Type B)", color="tab:red")
ax_comp_raw[2].plot(f_sim, np.rad2deg(s11_phase_sim), label="diag only (Type B)", color="tab:blue")

ax_comp_raw[3].semilogy(f_emp, np.rad2deg(s11_phase_unc_emp), label="statistical cov.", color="tab:gray", linewidth=4)
ax_comp_raw[3].semilogy(f_old, np.rad2deg(s11_phase_unc_old), label="diag only (Type B)", color="tab:red")

ax_comp_raw[0].legend()
ax_comp_raw[0].set_title("Frequency Domain")
ax_comp_raw[0].set_ylabel("magnitude [-]")
ax_comp_raw[1].set_ylabel("magnitude unc [-]")
ax_comp_raw[2].set_ylabel("phase [°]")
ax_comp_raw[3].set_ylabel("phase unc [°]")
ax_comp_raw[3].set_xlabel("f [GHz]")

## Preprocessing

### Data Loading

In [ ]:
# data to be used for further processing:
f, s11_ri, s11_ri_cov = utils.load_data("empirical_cov")
s11_mag, s11_phase, s11_mag_unc, s11_phase_unc = utils.convert_ri_cov_to_mag_phase_unc(s11_ri, s11_ri_cov)

### Apply Window in the Frequency Domain

This can be used to avoid effects of "hard edges" in the Fourier transform.

In [ ]:
# overall window shape
# window = signal.windows.get_window(("kaiser", 0.5 * np.pi), Nx=len(f), fftbins=False)
# "no window": 
window = np.ones(len(f))

# normalize so that integral of window is same as uniform window and therefore should preserve power of signal
window *= window.size / np.sum(window)

# window has zero uncertainty for now, but full covariance would be supported
window_cov = np.zeros((len(window), len(window)))
window_unc = np.sqrt(np.diag(window_cov))

# apply window
f_mod = f
s11_ri_mod, s11_ri_cov_mod = utils.apply_window(s11_ri, window, s11_ri_cov, window_cov)

### Zero Padding in the Frequency Domain

This can be used to (artificially) increase the resolution in the time domain.

In [ ]:
# will be done directly in iDFT + subsequent normalization

# pad_len = 1000
# df = np.mean(np.diff(f))
# f_ext = f[-1] + df * (1 + np.arange(pad_len))
# f_mod = np.r_[f, f_ext]
# s11_ri_mod = trimOrPad(s11_ri_mod, length=len(f) + pad_len, real_imag_type=True)
# s11_ri_cov_mod = trimOrPad(s11_ri_cov_mod, length=len(f) + pad_len, real_imag_type=True)

In [ ]:
s11_mag_mod, s11_phase_mod, s11_mag_unc_mod, s11_phase_unc_mod = utils.convert_ri_cov_to_mag_phase_unc(s11_ri_mod, s11_ri_cov_mod)

s11_phase_mod = np.nan_to_num(s11_phase_mod)
s11_mag_unc_mod = np.nan_to_num(s11_mag_unc_mod)
s11_phase_unc_mod = np.nan_to_num(s11_phase_unc_mod)

#### Visualize Input Data as Mag/Phase in the Frequency Domain

In [ ]:
# visualize raw input
fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True, sharex=True)

# plot
ax_in[0].plot(f_mod, s11_mag_mod, label="s11 windowed", color="tab:blue", linewidth=3)
ax_in[0].plot(f, s11_mag, label="s11", color="tab:gray")
ax_in[0].plot(f, window, label="window", color="tab:red")

ax_in[1].plot(f_mod, s11_mag_unc_mod, label="s11 windowed", color="tab:blue", linewidth=3)
ax_in[1].plot(f, s11_mag_unc, label="s11", color="tab:gray")

ax_in[2].plot(f_mod, np.rad2deg(s11_phase_mod), label="s11 windowed", color="tab:blue", linewidth=3)
ax_in[2].plot(f, np.rad2deg(s11_phase), label="s11", color="tab:gray")

ax_in[3].plot(f_mod, np.rad2deg(s11_phase_unc_mod), label="s11 windowed", color="tab:blue", linewidth=3)
ax_in[3].plot(f, np.rad2deg(s11_phase_unc), label="s11", color="tab:gray")

ax_in[0].legend()
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")

### Transformation into Time Domain (Application of Inverse DFT)

#### raw spectral data

In [ ]:
# convert unmodified reflection data to time domain
Nx = 2*len(s11_mag) - 1
S11, S11_cov = GUM_iDFT(s11_ri, s11_ri_cov, Nx=Nx)

# shift time signal
S11_shifted, S11_cov_shifted = shift_uncertainty(S11, S11_cov, shift=Nx//2)

# provide timestamps
# corresponds to:
#dt = 1 / (2*np.max(f_mod))   # frequency span is [-f_max , f_max]
t_span = 1 / np.mean(np.diff(f))  # original f, not f_mod
t = np.linspace(0, t_span, num=Nx)
t_shifted = t - t_span/2

#### modified (window + pad) spectral data

In [ ]:
# pad signal to increase time resolution
pad_len = 2500

# convert modified reflection data to time domain 
Nx_mod = Nx + pad_len
S11_mod, S11_cov_mod = GUM_iDFT(s11_ri_mod, s11_ri_cov_mod, Nx=Nx_mod)

# compensate the padding
S11_mod *= Nx_mod / Nx
S11_cov_mod *= Nx_mod / Nx

# shift time signal
S11_shifted_mod, S11_cov_shifted_mod = shift_uncertainty(S11_mod, S11_cov_mod, shift=Nx_mod//2)

# provide timestamps of modified data
t_mod = np.arange(0, Nx_mod) * (t[1] - t[0]) * Nx / float(Nx_mod) + t[0]  # from scipy resample
if len(t_mod) % 2 == 0:
    t_shifted_mod = t_mod - np.mean(t_mod[t_mod.size//2 - 1 : t_mod.size//2 + 1])
else:
    t_shifted_mod = t_mod - t_mod[t_mod.size//2]

#### visualize

In [ ]:
# visualize time domain data
fig_in_td, ax_in_td = plt.subplots(nrows=2, figsize=(8, 6), tight_layout=True, sharex=True)

ax_in_td[0].get_shared_x_axes().join(ax_in_td[0], ax_in_td[1])
ax_in_td[0].plot(t_shifted, np.abs(S11_shifted), label="S11", color="tab:green")
ax_in_td[0].plot(t_shifted_mod, np.abs(S11_shifted_mod), label="S11 mod", color="tab:gray")

ax_in_td[1].semilogy(t_shifted, np.sqrt(np.diag(S11_cov_shifted)), label="S11 unc", color="tab:green")
ax_in_td[1].semilogy(t_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod)), label="S11 mod unc", color="tab:gray")

ax_in_td[0].legend()
ax_in_td[0].set_title("Time Domain")
ax_in_td[0].set_ylabel("signal magnitude [-]")
ax_in_td[1].set_ylabel("signal unc [-]")
ax_in_td[1].set_xlabel("t [ns]")

In [ ]:
# visualize time domain data
fig_in_td_zoom, ax_in_td_zoom = plt.subplots(nrows=2, figsize=(8, 6), tight_layout=True, sharex=True)

ax_in_td_zoom[0].get_shared_x_axes().join(ax_in_td[0], ax_in_td[1])
#ax_in_td_zoom[0].plot(t_shifted, np.abs(S11_shifted), label="S11", color="tab:green")
ax_in_td_zoom[0].plot(t_shifted_mod, np.abs(S11_shifted_mod), label="S11 mod", color="tab:gray")

#ax_in_td_zoom[1].plot(t_shifted, np.sqrt(np.diag(S11_cov_shifted)), label="S11 unc", color="tab:green")
ax_in_td_zoom[1].plot(t_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod)), label="S11 mod unc", color="tab:gray")

ax_in_td_zoom[0].set_xlim((-0.1,0.65))

ax_in_td_zoom[0].legend()
ax_in_td_zoom[0].set_title("Time Domain")
ax_in_td_zoom[0].set_ylabel("signal magnitude [-]")
ax_in_td_zoom[1].set_ylabel("signal unc [-]")
ax_in_td_zoom[1].set_xlabel("t [ns]")

In [ ]:
if False:
    array_export = np.c_[t_shifted_mod, S11_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod))]
    df_export = pandas.DataFrame(array_export)
    df_export.columns = ["time", "signal", "signal_unc"]
    df_export.to_excel("export.xlsx")

In [ ]:
# visualize time domain data
fig_in_td_cov, ax_in_td_cov = plt.subplots(nrows=1, figsize=(6, 6), tight_layout=True)
maxi = np.max(np.abs(S11_cov))
cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=1e-14)

img0 = ax_in_td_cov.imshow(S11_cov_shifted_mod,  cmap="PuOr", norm=cnorm)

fig_in_td_cov.colorbar(img0, ax=ax_in_td_cov)
ax_in_td_cov.set_title("Covariance of (modified) time signal")

#### Check that Chaining iDFT + DFT Results in the Original Data

This shows that the low uncertainties in the time domain are just a consequence of 
the (inverse) Fourier transformation. 

In [ ]:
s11_ri_idtest, s11_ri_cov_idtest = GUM_DFT(S11, S11_cov)

print("(Numerical) identity is achieved for the real-imag-representation:")
print("real/imag representation identity: ", np.allclose(s11_ri_idtest, s11_ri))
print("Covariance identity:               ", np.allclose(s11_ri_cov_idtest, s11_ri_cov))


s11_mag_idtest, s11_phase_idtest, s11_UAP_idtest = DFT2AmpPhase(s11_ri_idtest, s11_ri_cov_idtest, tol=1e-15)
s11_mag_unc_idtest = np.sqrt(np.diag(s11_UAP_idtest)[:len(s11_mag_idtest)])
s11_phase_unc_idtest = np.sqrt(np.diag(s11_UAP_idtest)[len(s11_mag_idtest):])

print("\n\n(Numerical) identity is achieved for the mag-phase-representation: (except some elements with a mag/phase ratio close to zero)")
print("magnitude identity:     ", np.allclose(s11_mag_idtest, s11_mag))
print("magnitude unc identity: ", np.allclose(s11_mag_unc_idtest, s11_mag_unc))
print("phase identity:         ", np.allclose(s11_phase_idtest, s11_phase))
print("phase unc identity:     ", np.allclose(s11_phase_unc_idtest, s11_phase_unc))

## Define the Time-Gate

In [ ]:
gate_array_shifted_mod, gate_unc_array_shifted_mod = utils.gate(t_shifted_mod, t_start=0.0, t_end=0.18, kind="kaiser", order=2.5*np.pi)
gate_cov_array_shifted_mod = np.diag(np.square(gate_unc_array_shifted_mod))

# "unshift" for later calculations
gate_array_mod, gate_cov_array_mod = shift_uncertainty(gate_array_shifted_mod, gate_cov_array_shifted_mod, shift=-(Nx_mod-1)//2)

In [ ]:
fig_gate, ax_gate = plt.subplots(nrows=2, figsize=(8, 8), tight_layout=True, sharex=True)

# setup secondary y-axis
ax2_gate = [None, None]
ax2_gate[0] = ax_gate[0].twinx()
ax2_gate[1] = ax_gate[1].twinx()

# plot signal and gate
ax_gate[0].plot(t_shifted_mod, np.abs(S11_shifted_mod), label="S11 mod", color="tab:gray")
ax2_gate[0].plot(t_shifted_mod, gate_array_shifted_mod, label="gate", color="r")

# plot uncertainty of signal and gate
ax_gate[1].plot(t_shifted_mod, np.sqrt(np.diag(S11_cov_shifted_mod)), label="S11 mod unc", color="tab:gray")
ax2_gate[1].plot(t_shifted_mod, gate_unc_array_shifted_mod, label="gate unc", color="r")

# decorate
ax_gate[0].set_title("Time Domain (zoomed)")
ax_gate[0].legend(loc="upper right")
ax2_gate[0].legend(loc="upper center")
ax_gate[0].set_xlim((-0.1,1.5))
ax2_gate[1].set_ylim(ax_gate[1].get_ylim())

ax_gate[0].set_ylabel("signal magnitude [-]")
ax2_gate[0].set_ylabel("gate [-]")
ax_gate[1].set_ylabel("signal unc [-]")
ax2_gate[1].set_ylabel("gate unc [-]")
ax_gate[1].set_xlabel("t [ns]")

ax_gate[0].yaxis.get_label().set_color("tab:gray")
ax2_gate[0].yaxis.get_label().set_color("r")
ax_gate[1].yaxis.get_label().set_color("tab:gray")
ax2_gate[1].yaxis.get_label().set_color("r")

## Apply the Time-Gate to the modified Signal

### Method 1 (multiplication and then DFT)
Applies the gate already in the time-domain and converts the result back to frequency domain. 

In [ ]:
# main calls
S11_gated_mod = S11_mod * gate_array_mod
S11_gated_cov_mod = np.diag(gate_array_mod) @ S11_cov_mod @ np.diag(gate_array_mod).T + np.diag(S11_mod) @ gate_cov_array_mod @ np.diag(S11_mod).T
s11_gated_ri_mod, s11_gated_ri_cov_mod = GUM_DFT(S11_gated_mod, S11_gated_cov_mod)

In [ ]:
# undo zero-padding
s11_gated_ri = trimOrPad(s11_gated_ri_mod, length=len(f), real_imag_type=True) * Nx / Nx_mod
s11_gated_ri_cov = trimOrPad(s11_gated_ri_cov_mod, length=len(f), real_imag_type=True) * Nx / Nx_mod

# undo windowing
s11_gated_ri, s11_gated_ri_cov = utils.apply_window(s11_gated_ri, 1.0 / window, s11_gated_ri_cov, None)

#### Visualize Time Gated s11-Parameter

In [ ]:
# visualize result amp/phase
NN=len(s11_gated_ri)//2

# convert back to amplitude/phase representation
s11_gated_A, s11_gated_P, s11_gated_UAP = DFT2AmpPhase(s11_gated_ri, s11_gated_ri_cov)
s11_gated_UA = np.sqrt(np.diag(s11_gated_UAP)[:NN])
s11_gated_UP = np.sqrt(np.diag(s11_gated_UAP)[NN:])

fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_gated_A, label="s11 gated", color="tab:blue")
ax_in[1].plot(f, s11_gated_UA, label="s11 gated", color="tab:blue")
ax_in[2].plot(f, np.rad2deg(s11_gated_P), label="s11 gated", color="tab:blue")
ax_in[3].plot(f, np.rad2deg(s11_gated_UP), label="s11 gated", color="tab:blue")

ax_in[0].legend()
ax_in[0].grid(which="both", axis="both")
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")


### Method 2 (DFT and then complex convolution)

Transforms gate into frequency domain and applies gate to the original (frequency domain) signal by using convolution operation. Output should match first method up to numerical precision. (Done with uncertainty evaluation by Monte Carlo, analytical uncertainty evaluation out of scope for now.)

In [ ]:
def make_twosided(x):
    # returns the twosided spectrum with f=0 at the start (default numpy style)
    # x = x_re + 1j * x_im 
    x_twosided = np.r_[x, np.conjugate(x[1:][::-1])]  # odd signal length
    #x_twosided = np.r_[x, np.conjugate(x[::-1])]  # even signal length (default assumption for rfft)
    return x_twosided

def make_onesided(x):
    # returns the twosided spectrum with f=0 at the start (default numpy style)
    # x = x_re + 1j * x_im, (size = 2*N - 1)
    N = (x.size + 1) // 2   # odd signal length
    #N = x.size // 2   # even signal length
    x_onesided = x[:N]
    return x_onesided

def complex_convolution_of_two_half_spectra(X, Y):
    # complex valued X, Y

    # transform into full spectra
    XX = make_twosided(X)
    YY = make_twosided(Y)

    # otherwise not strict ascending order (numpy default has f=0 at index 0, not in the middle)
    XX = np.fft.fftshift(XX) 
    YY = np.fft.fftshift(YY)

    # actual convolution
    RR = convolve1d(XX, YY, mode="wrap") / XX.size

    # undo shifting and make half spectrum
    R = make_onesided(np.fft.ifftshift(RR))

    return R

In [ ]:
# main calls
#gate_spectrum = np.fft.rfft(gate_array)
#s11_gated_conv = complex_convolution_of_two_half_spectra(ri2c(s11_ri), gate_spectrum)

# Monte Carlo of this main call

# draw gate and signal
def draw_samples(size, x1, x1_cov, x2, x2_cov):
    SAMPLES_X1 = np.random.multivariate_normal(x1, x1_cov, size)
    SAMPLES_X2 = np.random.multivariate_normal(x2, x2_cov, size)
    return (SAMPLES_X1, SAMPLES_X2)

# evaluate
n_runs = 100
results = []
for s11_ri_mc, gate_array_mc in zip(*draw_samples(size=n_runs, x1=s11_ri_mod, x1_cov=s11_ri_cov_mod, x2=gate_array_mod, x2_cov=gate_cov_array_mod)):

    # explicitly zero pad
    s11_ri_mc = trimOrPad(s11_ri_mc, length=len(f) + pad_len//2, real_imag_type=True) * Nx_mod / Nx

    # main call
    gate_spectrum_tmp = np.fft.rfft(gate_array_mc)
    s11_gated_conv_tmp = complex_convolution_of_two_half_spectra(ri2c(s11_ri_mc), gate_spectrum_tmp)

    # undo zero-padding
    s11_gated_conv_tmp = trimOrPad(s11_gated_conv_tmp, length=len(f), real_imag_type=False) * Nx / Nx_mod

    # undo windowing
    s11_gated_conv_tmp = s11_gated_conv_tmp / window
    
    # save result
    results.append(c2ri(s11_gated_conv_tmp))

# extract mean and covariance
s11_gated_mcconv_ri = np.mean(results, axis=0)
s11_gated_mcconv_ri_cov = np.cov(results, rowvar=False)
s11_gated_mcconv = ri2c(s11_gated_mcconv_ri)

#### Visualize Time Gated s11-Parameter

In [ ]:
# visualize result amp/phase
NN=len(s11_gated_mcconv_ri)//2

# convert back to amplitude/phase representation
s11_gated_mcconv_A, s11_gated_mcconv_P, s11_gated_mcconv_UAP = DFT2AmpPhase(s11_gated_mcconv_ri, s11_gated_mcconv_ri_cov)
s11_gated_mcconv_UA = np.sqrt(np.diag(s11_gated_mcconv_UAP)[:NN])
s11_gated_mcconv_UP = np.sqrt(np.diag(s11_gated_mcconv_UAP)[NN:])

fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 8), tight_layout=True)
ax_in[0].plot(f, s11_gated_mcconv_A, label="s11 gated", color="tab:orange")
ax_in[1].plot(f, s11_gated_mcconv_UA, label="s11 gated", color="tab:orange")
ax_in[2].plot(f, np.rad2deg(s11_gated_mcconv_P), label="s11 gated", color="tab:orange")
ax_in[3].plot(f, np.rad2deg(s11_gated_mcconv_UP), label="s11 gated", color="tab:orange")

ax_in[0].legend()
ax_in[0].grid(which="both", axis="both")
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")

## Comparison

### Original and Time-Gated Spectra of both methods

In [ ]:
fig_in, ax_in = plt.subplots(nrows=4, figsize=(8, 10), tight_layout=True)

ax_in[0].plot(f, s11_mag, label="s11 orig", color="tab:gray")
ax_in[0].plot(f, s11_gated_A, label="s11 gated", color="tab:blue", linewidth=5)
ax_in[0].plot(f, np.abs(s11_gated_mcconv), label="s11 gated (MC conv)", color="tab:orange")

ax_in[1].plot(f, s11_mag_unc, label="s11 orig unc", color="tab:gray")
ax_in[1].plot(f, s11_gated_UA, label="s11 gated", color="tab:blue", linewidth=5)
ax_in[1].plot(f, s11_gated_mcconv_UA, label="s11 gated (MC conv)", color="tab:orange")

ax_in[2].plot(f, np.rad2deg(s11_phase), label="s11 orig", color="tab:gray")
ax_in[2].plot(f, np.rad2deg(s11_gated_P), label="s11 gated", color="tab:blue", linewidth=5)
ax_in[2].plot(f, np.rad2deg(np.angle(s11_gated_mcconv)), label="s11 gated (MC conv)", color="tab:orange")

ax_in[3].plot(f, np.rad2deg(s11_phase_unc), label="s11 orig unc", color="tab:gray")
ax_in[3].plot(f, np.rad2deg(s11_gated_UP), label="s11 gated", color="tab:blue", linewidth=5)
ax_in[3].plot(f, np.rad2deg(s11_gated_mcconv_UP), label="s11 gated (MC conv)", color="tab:orange")


ax_in[0].legend()
ax_in[1].legend()
ax_in[2].legend()
ax_in[3].legend()
ax_in[0].set_title("Frequency Domain")
ax_in[0].set_ylabel("magnitude [-]")
ax_in[1].set_ylabel("magnitude unc [-]")
ax_in[2].set_ylabel("phase [°]")
ax_in[3].set_ylabel("phase unc [°]")
ax_in[3].set_xlabel("f [GHz]")
ax_in[1].set_yscale("log")
ax_in[3].set_yscale("log")
ax_in[3].set_ylim(1e-3, 1.5e1)

### Covariance Matrices

Covariances in Re/Im Representation of:

- input data
- output of method 1 (analytical uncertainty evaluation)
- output of method 2 (Monte Carlo uncertainty evaluation)

In [ ]:
# utility function for annotating plots
def annotate_real_imag_plot(ax, annotate_x=True, annotate_y=True):
    # define new tick positions
    labels_re = [0, 10, 20, 30]  # GHz
    labels_re = [0, 8, 16, 24]  # GHz
    labels_im = labels_re
    labels = labels_re + labels_im

    # define new labels for these positions
    ticks_re = [np.flatnonzero(f == l).item() for l in labels_re]
    ticks_im = [f.size + 1 + k for k in ticks_re]
    ticks = ticks_re + ticks_im

    # define colors for the labels to distinguish real and imag parts
    colors_re = ["k"] * len(ticks_re)
    colors_im = ["r"] * len(ticks_im)
    tick_colors = colors_re + colors_im

    if annotate_x:
        # xticks (label, position, color)
        ax.set_xticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_xticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_xlabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(0.25, -0.01),
            xytext=(0.25, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(0.75, -0.01),
            xytext=(0.75, -0.10),
            fontsize=14,
            ha="center",
            va="bottom",
            xycoords="axes fraction",
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    if annotate_y:
        # yticks (label, position, color)
        ax.set_yticks(ticks=ticks, labels=labels)
        for ticklabel, c in zip(ax.get_yticklabels(), tick_colors):
            ticklabel.set_color(c)

        # axis label
        ax.set_ylabel("frequency [GHz]")

        # nice brace real part
        ax.annotate(
            "real",
            xy=(-0.01, 0.75),
            xytext=(-0.10, 0.75),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="black",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

        # nice brace imag part
        ax.annotate(
            "imag",
            xy=(-0.01, 0.25),
            xytext=(-0.10, 0.25),
            fontsize=14,
            ha="left",
            va="center",
            xycoords="axes fraction",
            rotation=90,
            color="red",
            #arrowprops=dict(arrowstyle="-[, widthB=6.0, lengthB=.5"),
        )

    return ax


In [ ]:
fig_cov, ax_cov = plt.subplots(nrows=3, figsize=(8, 25), tight_layout=True)
mini = max(1e-11, min(np.abs(s11_ri_cov).min(), np.abs(s11_gated_ri_cov).min()))
maxi = min(np.abs(s11_ri_cov).max(), np.abs(s11_gated_ri_cov).max())

cnorm = colors.SymLogNorm(vmin=-maxi, vmax=maxi, linthresh=mini)
img1 = ax_cov[0].imshow(s11_ri_cov, cmap="PuOr", norm=cnorm)
img2 = ax_cov[1].imshow(s11_gated_ri_cov, cmap="PuOr", norm=cnorm)
img3 = ax_cov[2].imshow(s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_cov.colorbar(img1, ax=ax_cov[0])
fig_cov.colorbar(img2, ax=ax_cov[1])
fig_cov.colorbar(img3, ax=ax_cov[2])

ax_cov[0].set_title("Covariance of s11_ri")
ax_cov[0] = annotate_real_imag_plot(ax_cov[0])

ax_cov[1].set_title("Covariance of s11_gated_ri")
ax_cov[1] = annotate_real_imag_plot(ax_cov[1])

ax_cov[2].set_title("Covariance of s11_gated_mcconv_ri_cov")
ax_cov[2] = annotate_real_imag_plot(ax_cov[2])

#### Compare Gated Spectra and Covariance Matrices of analytical and Monte Carlo approach

In [ ]:
# visual comparison of the two methods:
fig_comp, ax_comp = plt.subplots(nrows=3, figsize=(8, 20), tight_layout=True)

# mean signed difference of values
ax_comp[0].plot(s11_gated_ri - s11_gated_mcconv_ri)
ax_comp[0].set_title("Mean Signed Difference of Gated Spectra")
ax_comp[0] = annotate_real_imag_plot(ax_comp[0], annotate_y=False)

# mean signed difference of covariance matrices
img4 = ax_comp[1].imshow(s11_gated_ri_cov - s11_gated_mcconv_ri_cov, cmap="PuOr", norm=cnorm)
fig_comp.colorbar(img4, ax=ax_comp[1])

ax_comp[1].set_title("Signed Difference of both Covariance Matrices")
ax_comp[1] = annotate_real_imag_plot(ax_comp[1])

# Kullback-Leibler divergence of covariance matrices
kl_div = special.kl_div(s11_gated_ri_cov, s11_gated_mcconv_ri_cov)
cnorm_kl = colors.LogNorm(vmin=1e-12, vmax=1e-8, clip=True)

img5 = ax_comp[2].imshow(kl_div, cmap="binary", norm=cnorm_kl)
fig_comp.colorbar(img5, ax=ax_comp[2])


ax_comp[2].set_title("Kullback-Leibler divergence of both Covariance Matrices")
ax_comp[2] = annotate_real_imag_plot(ax_comp[2])